In [1]:
%use kotlin-statistics, krangl, kravis, klaxon

In [2]:
import kotlin.math.*

In [3]:
@file:Repository("https://dl.bintray.com/kyonifer/maven")
@file:DependsOn("com.kyonifer:koma-core-ejml:0.12")
@file:DependsOn("com.kyonifer:koma-plotting:0.12")

In [4]:
import koma.extensions.*
import koma.*

In [5]:
val df = DataFrame.readTSV("../data/UI_1-2buckets_6march_filt.tsv")

In [20]:
df

device_id,time_epoch,event_id,group_id,data
1312191092f7f25-dafb-4872-bdef-48d862040e85,1583453298421,finished,completion,finished
2204192da927d8d-1396-4efd-8e1a-18b59ebf81c6,1583472538347,action.invoked,actions,com.intellij.codeInsight.daemon.impl.DaemonTool...
25101933e9a2c4e-9cdc-4acc-9688-e7b99555246d,1583504322229,close,ui.dialogs,com.intellij.openapi.fileChooser.ex.FileChooser...
25101933e9a2c4e-9cdc-4acc-9688-e7b99555246d,1583505808237,frame.activated,lifecycle,frame.activated
0505191d5ca0393-2e1d-4356-b3d2-afba2ab75a37,1583506904562,action.invoked,actions,SaveAll
0705193eb2dae1e-9342-408a-8ab1-bac507bad3eb,1583479655202,edit,file.types.usage,JAVA
0705193eb2dae1e-9342-408a-8ab1-bac507bad3eb,1583479808124,action.invoked,actions,EditorCompleteStatement
0705193eb2dae1e-9342-408a-8ab1-bac507bad3eb,1583480452798,fetch.started,vcs,fetch.started
1312191092f7f25-dafb-4872-bdef-48d862040e85,1583453297837,finished,completion,finished
0505191d5ca0393-2e1d-4356-b3d2-afba2ab75a37,1583507129110,close,ui.dialogs,com.intellij.execution.actions.ChooseRunConfigu...


In [8]:
class Action(val time: Long, val event_id: String, val group_id: String, val event_data: String) {
    fun print() {
        println("$time $event_id $group_id $event_data")
    }
    
    fun get_event(): String {
        return group_id + "_" + event_data
    }
}

In [9]:
class Session(val id: String) {
    public var actions: ArrayList<Action> = ArrayList()

    fun add(time: Long, event: String, group: String, action: String) {
        actions.add(Action(time, event, group, action))
    }

    fun print() {
        println(id)
        for (action in actions) {
            action.print()
        }
    }
    
    fun time(): Long {
        val t: Long = actions[actions.size-1].time
        return t / 100 / 60
    }
    
    fun events(): String {
        val events = ArrayList<String>()
        for (action in actions)
            events.add(action.get_event())
        return events.joinToString(separator = " , ")
    }
}

In [46]:
val vcs_sessions = ArrayList<Session>()
val vcs_time1 = ArrayList<Long>()
val vcs_time2 = ArrayList<Long>()

In [47]:
fun makeSessions(path: String, threshold: Long = 60000): ArrayList<Session> {
    val df = DataFrame.readTSV(path)
    val groups = df.groupBy("device_id").sortedBy("time_epoch").groups()
    val ses: ArrayList<Session> = ArrayList()
    var tot = -1
    for (group in groups) {
        var flag = false
        var cnt: Int = -1
        var prev: Long = -1
        var start: Long = -1
        // ses.add(Session(groups[0].row(0)['device_id']))
        for (row in group.rows) {
            
            
            
            val dev = row.getOrDefault("device_id", "") as String
            val time = row.getOrDefault("time_epoch", 0) as Long
            val event = row.getOrDefault("event_id", "") as String
            val groupId = row.getOrDefault("group_id", "") as String
            val ed = row.getOrDefault("data", "") as String
            
        
            if (event == "fetch.finished")
                flag = true
            
            
            if (groupId == "lifecycle")
                if (event == "frame.activated" || event == "frame.deactivated")
                    continue
            
            
            if (flag && prev != -1L) {
                    vcs_time2.add(time-prev)
            }
            
            
            if (prev == -1L || time - prev > threshold) {
                if (flag && prev != -1L) {
                    vcs_sessions.add(ses[tot])
                    vcs_time1.add(time-prev)
                }
                cnt++
                tot++
                start = time
                ses.add(Session(dev + "_" + cnt.toString()))
                ses[tot].add(time - start, event, groupId, ed)
            } else {
                ses[tot].add(time - start, event, groupId, ed)
            }
            
            prev = time
            flag = event == "fetch.finished"
            

        }
    }
    return ses
}



In [48]:
val ses = makeSessions("../data/UI_1-2buckets_6march_filt.tsv")

In [49]:
ses.size

19865

In [50]:
vcs_sessions.size

947

In [59]:
var time1 = 1.0 *vcs_time1.sum() / vcs_time1.size / 100 / 60

In [60]:
var time2 = 1.0 *vcs_time2.sum() / vcs_time2.size / 100 / 60

In [66]:
var time3 = (vcs_time2.size * time2 - vcs_time1.size * time1) / (vcs_time2.size - vcs_time1.size)

In [75]:
"среднее время между между fetch.finished и следующим действием, если fetch последний в сессии = $time1 мин"

среднее время между между fetch.finished и следующим действием, если fetch последний в сессии = 95.96809116508273 мин

In [76]:
"среднее время между между fetch.finished и следующим действием = $time2 мин"

среднее время между между fetch.finished и следующим действием = 27.701385248354274 мин

In [77]:
"среднее время между между fetch.finished и следующим действием в данной сессии= $time3 мин"

среднее время между между fetch.finished и следующим действием в данной сессии= 0.7082451635351371 мин

In [78]:
val vcs_len = ArrayList<Long>()
for (vcs_session in vcs_sessions)
    vcs_len.add(vcs_session.time())

In [82]:
var time4 = 1.0 * vcs_len.sum() / vcs_len.size / 100

In [83]:
"средняя длина сессии с fetch.finished = $time4 сек"

средняя длина сессии с fetch.finished = 0.01775079197465681 сек